In [1]:
import numpy as np
import pandas as pd
import matplotlib
import os
from keras.layers import Input, BatchNormalization, Concatenate, Dense, GlobalAveragePooling2D, Dropout, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras import regularizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History, TensorBoard
from keras.layers import Embedding, Flatten, Dense
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
D = 100 # embedding dimension
H = 300 # no.of hidden layer nodes

In [37]:
product_desc = np.load('train_desc.npy')
product_query = np.load('train_query.npy')
product_desc_test = np.load('test_desc.npy')
product_query_test = np.load('test_query.npy')
train_image_embeddings = np.load('train_image_features.npy')
test_image_embeddings = np.load('test_image_features.npy')
product_id_final_test = np.load('product_id_final_test.npy')

In [4]:
train_image_embeddings.shape

(5659, 1000)

In [5]:
final_text = np.concatenate((product_query, product_query_test),axis=0)

In [6]:
import numpy as np
modelGlove = {}
with open("glove.6B.100d.txt", "r", encoding='utf-8') as lines:
        line = lines.readline()
        while line:
            modelGlove[line.split()[0]] = np.array(list(map(float, line.split()[1:])))
            line = lines.readline()

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(final_text)
sequences_query = tokenizer.texts_to_sequences(product_query)
sequence_desc = tokenizer.texts_to_sequences(product_desc)
sequences_query_test = tokenizer.texts_to_sequences(product_query_test)
sequences_desc_test = tokenizer.texts_to_sequences(product_desc_test)

/home/jharsh/miniconda3/envs/kerasGPU1/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 476 unique tokens.


In [9]:
data_d = pad_sequences(sequence_desc, maxlen=300)

In [10]:
data_d_test = pad_sequences(sequences_desc_test, maxlen=300)

In [11]:
data_q = pad_sequences(sequences_query, maxlen=20)

In [12]:
data_q_test = pad_sequences(sequences_query_test, maxlen=20)

In [13]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [14]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = modelGlove.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [15]:
from keras.layers import Embedding

embedding_layer_q = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=20,
                            trainable=False)
embedding_layer_d = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=300,
                            trainable=False)

In [16]:
D = 100 # embedding dimension
H = 300 # no.of hidden layer nodes

In [17]:
# sequence_input_d = Input(shape=(300,), dtype='int32')
# embeddings_d = embedding_layer_d(sequence_input_d)
sequence_input_q = Input(shape=(20,), dtype='int32')
embeddings_q = embedding_layer_q(sequence_input_q)
imageEmbeddingInput = Input(shape=(1000,))
x4 = Dense(500,activation='relu')(imageEmbeddingInput)
# embedding_layer_d_flatten = Flatten()(embeddings_d) 
embedding_layer_q_flatten = Flatten()(embeddings_q)
x3 = Dense(H,activation='relu')(x4)
# x1 = Dense(H,activation='relu')(embedding_layer_d_flatten)
x2 = Dense(H,activation='relu')(embedding_layer_q_flatten)
merge_x = concatenate([x2,x3])
x4 = Dense(H,activation='relu')(merge_x)
x4 = Dense(H//4,activation='relu')(x4)
x4 = Dense(H//16,activation='relu')(x4)
#x4 = Dropout(0.5)(x4)
final_x = Dense(1)(x4)

In [18]:
model = Model(inputs=[sequence_input_q,imageEmbeddingInput], outputs=final_x)


In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 100)      47700       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2000)         0           embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (D

In [20]:
plot_model(model, to_file='model.png')

In [21]:
model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.0001),
              metrics=['mse','mae'])

In [22]:
train_relevance = np.load('final_train_Y.npy')

In [23]:
test_relevance = np.load('final_test_Y.npy')

In [24]:
test_relevance.shape

(1544,)

In [25]:
train_relevance.shape

(5659,)

In [26]:
from sklearn.utils import shuffle
train_query, train_image_embedding_model, train_relevance = shuffle(data_q, train_image_embeddings, train_relevance)

In [27]:
train_query.shape, train_relevance.shape, train_image_embedding_model.shape

((5659, 20), (5659,), (5659, 1000))

In [28]:
train_valid_split = 5000

In [29]:
valid_query = train_query[train_valid_split:]
valid_relevance = train_relevance[train_valid_split:]
valid_image = train_image_embedding_model[train_valid_split:]

In [30]:
valid_image.shape


(659, 1000)

In [31]:
train_query = train_query[:train_valid_split]
train_relevance = train_relevance[:train_valid_split]
train_image_embedding_model = train_image_embedding_model[:train_valid_split]

In [32]:
checkpointer = ModelCheckpoint(filepath='NN_best_1.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_lr=0)
earlystopping = EarlyStopping(monitor='val_loss', patience=20)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [33]:
model.fit([train_query, train_image_embedding_model],train_relevance, epochs=100, verbose=2,
          validation_data=([valid_query, valid_image], valid_relevance), batch_size=16,
                    callbacks=[checkpointer, earlystopping])

Train on 5000 samples, validate on 659 samples
Epoch 1/100
 - 7s - loss: 1.0202 - mean_squared_error: 1.3508 - mean_absolute_error: 0.8597 - val_loss: 0.8017 - val_mean_squared_error: 0.6709 - val_mean_absolute_error: 0.6582

Epoch 00001: val_loss improved from inf to 0.80168, saving model to NN_best_1.hdf5
Epoch 2/100
 - 5s - loss: 0.7225 - mean_squared_error: 0.5438 - mean_absolute_error: 0.5704 - val_loss: 0.7985 - val_mean_squared_error: 0.6591 - val_mean_absolute_error: 0.6702

Epoch 00002: val_loss improved from 0.80168 to 0.79847, saving model to NN_best_1.hdf5
Epoch 3/100
 - 4s - loss: 0.7021 - mean_squared_error: 0.5151 - mean_absolute_error: 0.5540 - val_loss: 0.7547 - val_mean_squared_error: 0.5980 - val_mean_absolute_error: 0.5742

Epoch 00003: val_loss improved from 0.79847 to 0.75467, saving model to NN_best_1.hdf5
Epoch 4/100
 - 5s - loss: 0.6890 - mean_squared_error: 0.4977 - mean_absolute_error: 0.5402 - val_loss: 0.7618 - val_mean_squared_error: 0.6072 - val_mean_abso

In [34]:
model.load_weights('NN_best_1.hdf5')

In [35]:
model.evaluate([data_q_test,test_image_embeddings],test_relevance)

1544/1544 [==============================] - 0s 115us/step


[0.7401169408788335, 0.5740870790160382, 0.5616826368737097]

In [36]:
predictedResilts = model.predict([data_q_test,test_image_embeddings])

In [43]:
resDumpRawImage = []
for i in range(len(predictedResilts)):
    resDumpRawImage.append((product_query_test[i],product_id_final_test[i],test_relevance[i], np.asscalar(predictedResilts[i])))
    

In [44]:
resDumpRawImage[0]

('16gb memory card', 711162616, 4.0, 3.974149465560913)

In [45]:
np.save('resDumpRawImage', resDumpRawImage)